In [1]:
import sqlite3
import pandas as pd

In [20]:
db_old = sqlite3.connect("iceform.db")
db_old_cur = db_old.cursor()

db_new = sqlite3.connect("../../ice_site/ice_site.db")
db_new_cur = db_new.cursor()

**token**

In [4]:
db_old_cur.execute("SELECT id, wordform, lemma FROM token")
data = db_old_cur.fetchall()
db_new_cur.executemany("INSERT INTO token VALUES (?, ?, ?)", data)
db_new.commit()

**text_content**

In [5]:
df = pd.read_sql_query("""
SELECT tc.id, tc.text_id, ts.ch_idx, ts.p_idx, ts.s_idx, tc.sentence_id, tc.idx, token_id
FROM text_content tc
JOIN text_structure ts ON ts.text_id = tc.text_id AND ts.sentence_id = tc.sentence_id
""", con=db_old)
uniques = df.groupby(["text_id", "sentence_id"], as_index=False).agg({"id": "count"})
uniques["id"] = list(range(1, uniques.shape[0] + 1))
uniques.columns = ["text_id", "sentence_id", "sentence_unique"]
df = df.merge(uniques)
df = df[["id", "text_id", "ch_idx", "p_idx", "s_idx", "sentence_unique", "idx", "token_id"]]
db_new_cur.executemany("INSERT INTO text_content VALUES (?, ?, ?, ?, ?, ?, ?, ?)", df.values.tolist())
db_new.commit()

In [10]:
df = pd.read_sql_query("""
SELECT `index`, text_id, chapter, paragraph, sentence, start, end, short_ngram_id, cluster
FROM topic_ngram_final
""", con=db_old)
df.head()

,index,text_id,chapter,paragraph,sentence,start,end,short_ngram_id,cluster
0,0,7,44,21,4,0,3,5,0
1,1,14,67,3,1,11,14,5,0
2,2,47,296,5,4,1,4,5,0
3,3,47,296,5,4,1,3,5,0
4,4,47,469,2,4,0,3,5,0


**cluster_filters**

In [11]:
df = pd.read_sql_query("""
SELECT ngram_by_text_final.short_ngram_id, ngram_by_text_final.cluster, count(`index`) as n_entries, count(distinct text_id) as uq_text, textual.text
FROM ngram_by_text_final
JOIN (
	SELECT t.short_ngram_id, t.cluster, group_concat(tk.wordform, " ") as text
	FROM (
	SELECT short_ngram_id, cluster, unique_ngram_id, count(`index`) as cnt, text_id, chapter, paragraph, sentence, start, end
	FROM topic_ngram_final
	GROUP BY short_ngram_id, cluster
	ORDER BY short_ngram_id, cluster, cnt DESC, unique_ngram_id) as t
	JOIN text_structure as ts ON t.text_id = ts.text_id AND t.chapter = ts.ch_idx AND t.paragraph = ts.p_idx AND t.sentence = ts.s_idx
	JOIN text_content as tc ON tc.text_id = ts.text_id AND ts.sentence_id = tc.sentence_id
	JOIN token as tk ON tc.token_id = tk.id
	WHERE t.start <= tc.idx AND t.end >= tc.idx
	GROUP BY t.short_ngram_id, t.cluster
) as textual ON ngram_by_text_final.short_ngram_id = textual.short_ngram_id AND ngram_by_text_final.cluster = textual.cluster
GROUP BY ngram_by_text_final.short_ngram_id, ngram_by_text_final.cluster
ORDER BY ngram_by_text_final.short_ngram_id, ngram_by_text_final.cluster""", con=db_old)

In [12]:
uniques = df.groupby(["short_ngram_id", "cluster"], as_index=False).agg({"n_entries": "count"})
uniques["n_entries"] = list(range(1, uniques.shape[0] + 1))
uniques.columns = ["short_ngram_id", "cluster", "cluster_id"]
df = df.merge(uniques)

In [13]:
df = df.reset_index()
df.head()

,index,short_ngram_id,cluster,n_entries,uq_text,text,cluster_id
0,0,2,4,4,4,sperði eg þig ekki,1
1,1,4,1,4,4,því að þeir þoldu,2
2,2,4,4,4,4,áfram svo að hann kastar henni upp,3
3,3,4,25,3,2,þá hann fréttir því,4
4,4,4,29,3,3,því að þeir deildu um,5


In [14]:
db_new_cur.executemany("""
INSERT INTO cluster_filters (id, short_ngram_id, cluster, n_entries, unique_text, text, cluster_id)
VALUES (?, ?, ?, ?, ?, ?, ?)
""", df.values.tolist())
db_new.commit()

**ngram_entries**

In [30]:
df = pd.read_sql_query("""
SELECT short_ngram_id, cluster, text_id as text, paragraph, chapter, sentence, start, end
FROM ngram_by_text_final
""", con=db_old)

df.head()

,short_ngram_id,cluster,text,paragraph,chapter,sentence,start,end
0,2,4,0,21,5,2,1,4
1,2,4,7,12,27,1,14,17
2,2,4,17,1,4,1,1,3
3,2,4,38,14,2,2,6,8
4,4,1,8,4,28,4,7,10


In [31]:
txt = pd.read_sql_query("""SELECT distinct text, chapter, paragraph, sentence, sentence_unique FROM text_content""", con=db_new)
clust = pd.read_sql_query("""SELECT short_ngram_id, cluster_id FROM cluster_filters""", con=db_new)

df = df.merge(txt).merge(clust)

df = df[["short_ngram_id", "cluster_id", "text", "chapter", "paragraph", "sentence", "sentence_unique", "start", "end"]].reset_index()
df.columns = ["id", "short_ngram_id", "cluster_id", "text", "chapter", "paragraph", "sentence", "sentence_unique", "start", "end"]
df.head()

,id,short_ngram_id,cluster_id,text,chapter,paragraph,sentence,sentence_unique,start,end
0,0,2,1,0,5,21,2,281,1,4
1,1,2,1,7,27,12,1,16094,14,17
2,2,2,1,17,4,1,1,33113,1,3
3,3,2,1,38,2,14,2,56271,6,8
4,4,4,2,8,28,4,4,18685,7,10


In [32]:
# db_new_cur.executemany("INSERT INTO ngram_entries VALUES (?, ?, ?, ?, ?, ?, ?)", df.values.tolist())
# db_new.commit()
df.to_sql("ngram_entries", con=db_new, index=False, if_exists="replace")

**text**

In [25]:
df = pd.read_sql_query("""SELECT id, text_name FROM text""", con=db_old)

df.to_sql("text", con=db_new, index=False, if_exists="replace")